In [47]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import os
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm 
from datetime import date 
from datetime import datetime
from collections import defaultdict 

In [48]:
# from foodparser import foodparser

In [49]:
# fp = foodparser.FoodParser()

In [3]:
# !pip install pyspellchecker

%load_ext autoreload
%autoreload 2

# %run '/content/drive/My Drive/Colab Data/public_app_analysis/utils/publicAnalysisHelper.py'
# %run ../utils/publicAnalysisHelper.py
# %run ../food_parser/food_parser.py

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
# from google.colab import drive
# drive.mount('/content/drive')
file_dir = '../'

data_dir = file_dir + 'data/'
result_dir = file_dir + 'results/'

In [52]:
fp = FoodParser()

fp.parse_food('Espresso', return_sentence_tag = True)

(['espresso'], 1, 0, [])

# Merge combine_df files

In [6]:
# gram_set = pd.read_csv('../food_parser/final_database/gram_set.csv')
# gram_tag = pd.read_csv('../food_parser/final_database/gram_tag.csv')
# gram_set.shape, gram_tag.shape

In [7]:
# gram_set.drop(columns = ['gram_type']).merge(gram_tag, on = 'gram_key', how = 'outer')\
#     .to_csv('../food_parser/final_database/combined_gram_set.csv', index = False)

In [8]:
# gram_tag.drop(columns = ['gram_type']).merge(gram_set, on = 'gram_key')

# Load a sample of data

In [9]:
ff_sample_pickle_file = open(data_dir + 'fire_fighter/FFFoodData_Unique_Time_All_Aug2020.pickle', 'rb')
ff_sample = pickle.load(ff_sample_pickle_file).head(100000)
ff_sample.head(2)

,PID,caloric,day,desc_text,food_type,local_time,original_logtime,original_logtime_notz,participant,study,time,tz
1,9bbi18050004,1,2018-07-18,kind bar && coffee,f,9.583333,07-18-2018 09:35AM PDT,2018-07-18 09:35:00,Hero-001,TRE,09:35:00,PDT
2,9bbi18050004,1,2018-06-24,potatoe,f,19.116667,06-24-2018 07:07PM PDT,2018-06-24 19:07:00,Hero-001,TRE,19:07:00,PDT


In [10]:
fp = FoodParser()

# fp.all_gram_set[0].add('yoghurt')

'yoghurt' in fp.all_gram_set[0]

True

In [11]:
sample_df = ff_sample
if 'ID' not in sample_df:
    sample_df['ID'] = range(sample_df.shape[0])

total_unknown = 0
total_token = 0
parsed = []
total_unknown_tokens = []
for i in tqdm(range(sample_df.shape[0])):
    entry = sample_df.iloc[i].desc_text
    result, num_token, num_unknown, unknown_tokens = fp.parse_food(entry, return_sentence_tag = True)
    result = [(f, fp.find_food_type(f)) for f in result]
    parsed.append(result)
    total_token += num_token
    total_unknown += num_unknown
    total_unknown_tokens += unknown_tokens

100%|██████████| 100000/100000 [08:01<00:00, 207.88it/s]


- parsed

In [12]:
sample_df['parsed'] = parsed

# Build search functions

## Look-up-dict

In [14]:
gram_tag_df = pd.read_csv('../food_parser/final_database/combined_gram_set.csv').fillna('empty')\
    [['gram_type', 'food_type', 'gram_key', 'tag1', 'tag2', 'tag3', 'tag4']]
gram_tag_df.head()

,gram_type,food_type,gram_key,tag1,tag2,tag3,tag4
0,1,f,chilli,empty,empty,empty,empty
1,1,f,chile,empty,empty,empty,empty
2,1,f,quinoa,empty,empty,empty,empty
3,1,m,magnesium,??,empty,empty,empty
4,1,m,multivitamin,empty,empty,empty,empty


In [15]:
def basic_clean(s):
    tmp_s = s.lower().replace('?', '')
    if len(tmp_s) == 0:
        return 'empty'
    else:
        return tmp_s
gram_tag_df.tag1 = gram_tag_df.tag1.apply(basic_clean)
gram_tag_df.tag2 = gram_tag_df.tag2.apply(basic_clean)
gram_tag_df.tag3 = gram_tag_df.tag3.apply(basic_clean)

In [16]:
gram_tag_df.head()

,gram_type,food_type,gram_key,tag1,tag2,tag3,tag4
0,1,f,chilli,empty,empty,empty,empty
1,1,f,chile,empty,empty,empty,empty
2,1,f,quinoa,empty,empty,empty,empty
3,1,m,magnesium,empty,empty,empty,empty
4,1,m,multivitamin,empty,empty,empty,empty


In [17]:
look_up_dict = {}
for row_i in range(gram_tag_df.shape[0]):
    tmp_lst = []
    for item in gram_tag_df.iloc[row_i].values[2:]:
        if item == 'empty':
            break
        tmp_lst.append(item)
    look_up_dict[tmp_lst[0]] = tmp_lst

In [18]:
gram_tag_df.iloc[0].values

array([1, 'f', 'chilli', 'empty', 'empty', 'empty', 'empty'], dtype=object)

In [19]:
[i for i in list(look_up_dict.items())[:3]]

[('chilli', ['chilli']), ('chile', ['chile']), ('quinoa', ['quinoa'])]

## One-hot-dict

In [20]:
counter = 0
idx_to_tag = {}
tag_to_idx = {}
for key in look_up_dict.keys():
    for tag in look_up_dict[key]:
        if tag not in tag_to_idx.keys():
            idx_to_tag[counter] = tag
            tag_to_idx[tag] = counter
            counter += 1

In [21]:
all_tag_arr = np.array(list(tag_to_idx.keys()))

In [22]:
parsed[:3] # This is the array we need to analysis

[[('kind bar', 'f'), ('coffee', 'b')],
 [('potato', 'f')],
 [('egg', 'f'), ('sausage', 'f')]]

In [23]:
vec_size = counter
def encode_single_item(item):
    tags = look_up_dict[item]
    return [tag_to_idx[t] for t in tags]

def encode_single_parsed_entry(result_lst):
    encoded_arr = np.array([0] * vec_size)
    for pair in result_lst:
        item = pair[0]
        encoded_arr[encode_single_item(item)] = 1
    return encoded_arr

def decode_item_arr(arr):
    return all_tag_arr[arr == 1]

def find_str_contain(df, s):
    return df.loc[df.desc_text.apply(lambda x: s in x)].reset_index(drop = True)

In [24]:
vec_size

981

In [25]:
parsed_encoded_arrays = []
parsed_tagged_text = []
for p in parsed:
    encoded_arr = encode_single_parsed_entry(p)
    parsed_encoded_arrays.append(encoded_arr)
    parsed_tagged_text.append(decode_item_arr(encoded_arr))

parsed_encoded_arrays = np.array(parsed_encoded_arrays)

In [26]:
parsed_tagged_text[:15]

[array(['coffee', 'kind bar'], dtype='<U39'),
 array(['potato'], dtype='<U39'),
 array(['sausage', 'egg'], dtype='<U39'),
 array(['kind bar'], dtype='<U39'),
 array(['salad', 'chicken pot pie'], dtype='<U39'),
 array(['potato', 'steak'], dtype='<U39'),
 array(['sausage'], dtype='<U39'),
 array(['avocado', 'salad'], dtype='<U39'),
 array(['cake', 'yogurt', 'pancake', 'blueberry'], dtype='<U39'),
 array(['fruit', 'yogurt', 'apple'], dtype='<U39'),
 array(['coffee', 'banana'], dtype='<U39'),
 array(['elk', 'baked bean'], dtype='<U39'),
 array(['kind bar'], dtype='<U39'),
 array(['avocado', 'burrito'], dtype='<U39'),
 array(['pizza'], dtype='<U39')]

In [27]:
# encode_single_item('fish taco')

In [28]:
parsed_encoded_arrays.shape

(100000, 981)

In [29]:
parsed_encoded_arrays[:, 1].shape

(100000,)

In [30]:
sample_df.shape

(100000, 14)

In [31]:
len(parsed)

100000

In [32]:
def search(encoded_arr, tag, id_lst):
    idx = tag_to_idx[tag]
    selected_entry_spots = id_lst[encoded_arr[:, idx] != 0]
    return selected_entry_spots

def backward_search(tag):    
    return [k for k in look_up_dict.keys() if tag in look_up_dict[k]]

In [33]:
backward_search('coffee')[:3]

['mocha', 'latte', 'cappuccino']

## Deploy on Coffee example

In [34]:
coffee_found = search(parsed_encoded_arrays, 'coffee', np.array(range(100000)))
coffee_spot = sample_df.reset_index(drop = True).loc[list(coffee_found)].reset_index(drop = True)

In [35]:
coffee_spot.shape

(8833, 14)

In [36]:
find_str_contain(coffee_spot, 'espresso').head()

,PID,caloric,day,desc_text,food_type,local_time,original_logtime,original_logtime_notz,participant,study,time,tz,ID,parsed
0,9bbi18060029,1,2018-07-17,espresso,b,14.750000,07-17-2018 02:45PM GMT-07:00,2018-07-17 14:45:00,Hero-016,TRE,14:45:00,GMT-07:00,12326,"[(espresso, b)]"
1,9bbi18070041,1,2019-03-22,"coconut pb perfect bar && triple espresso, alm...",f,8.050000,03-22-2019 08:03AM EDT,2019-03-22 08:03:00,Hero-028,TRE,08:03:00,EDT,20579,"[(coconut peanut butter bar, f), (espresso, b)..."
2,9bbi18070041,1,2019-01-29,"breakfast burrito && whip cream, triple espresso",f,9.250000,01-29-2019 09:15AM PST,2019-01-29 09:15:00,Hero-028,TRE,09:15:00,PST,21315,"[(breakfast burrito, f), (cream, b), (espresso..."
3,9bbi18070041,1,2019-03-22,"triple espresso, almond milk 2 oz",b,11.566667,03-22-2019 11:34AM EDT,2019-03-22 11:34:00,Hero-028,TRE,11:34:00,EDT,21788,"[(espresso, b), (almond milk, b)]"
4,9bbi18070041,1,2019-05-22,"double espresso, almond milk 2 oz",b,14.000000,05-22-2019 02:00PM PDT,2019-05-22 14:00:00,Hero-028,TRE,14:00:00,PDT,21793,"[(espresso, b), (almond milk, b)]"


In [37]:
# for k in list(look_up_dict.keys()):
#     if 'coffee' in look_up_dict[k]:
#         print(look_up_dict[k])

In [38]:
find_str_contain(sample_df, 'latte')

,PID,caloric,day,desc_text,food_type,local_time,original_logtime,original_logtime_notz,participant,study,time,tz,ID,parsed
0,9bbi18050005,1,2018-06-05,iced vanilla latte,f,15.550000,06-05-2018 03:33PM PDT,2018-06-05 15:33:00,Hero-002,TRE,15:33:00,PDT,1379,"[(vanilla, f)]"
1,9bbi18060014,1,2018-06-30,cereal witb berries milk && latte with 1 sugar...,f,8.483333,06-30-2018 08:29AM GMT-07:00,2018-06-30 08:29:00,Hero-007,SOC,08:29:00,GMT-07:00,4078,"[(sugar syrup, b), (cereal, f), (berry, f), (m..."
2,9bbi18060014,1,2018-07-20,cereal with berries and yogurt && latte with 1...,f,8.166667,07-20-2018 08:10AM GMT-07:00,2018-07-20 08:10:00,Hero-007,SOC,08:10:00,GMT-07:00,4083,"[(sugar syrup, b), (cereal, f), (berry, f), (y..."
3,9bbi18060014,1,2018-07-08,cereal with berries and yogurt && latte with 1...,f,7.350000,07-08-2018 07:21AM GMT-07:00,2018-07-08 07:21:00,Hero-007,SOC,07:21:00,GMT-07:00,4090,"[(sugar syrup, b), (cereal, f), (berry, f), (y..."
4,9bbi18060014,1,2018-08-04,cereal with berries and yogurt && latte with 1...,f,7.200000,08-04-2018 07:12AM GMT-07:00,2018-08-04 07:12:00,Hero-007,SOC,07:12:00,GMT-07:00,4106,"[(sugar syrup, b), (cereal, f), (berry, f), (y..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,9bbi19080141,1,2020-01-30,ned vanilla oat milk latte,b,12.366667,01-30-2020 12:22PM PST,2020-01-30 12:22:00,Hero-122,TRE,12:22:00,PST,96040,"[(vanilla, f), (oat, f), (milk, b)]"
388,9bbi19080141,1,2020-02-05,oat milk latte,b,9.916667,02-05-2020 09:55AM PST,2020-02-05 09:55:00,Hero-122,TRE,09:55:00,PST,96174,"[(oat, f), (milk, b)]"
389,9bbi19080141,1,2019-10-30,almond milk latte,b,17.733333,10-30-2019 05:44PM PDT,2019-10-30 17:44:00,Hero-122,TRE,17:44:00,PDT,96191,"[(almond milk, b)]"
390,9bbi19080141,1,2020-02-06,small cinnamon oat milk latte,b,12.716667,02-06-2020 12:43PM PST,2020-02-06 12:43:00,Hero-122,TRE,12:43:00,PST,96211,"[(cinnamon, f), (oat, f), (milk, b)]"


In [39]:
look_up_dict['coffee']

['coffee']

In [40]:
coffee_spot.head(2)

,PID,caloric,day,desc_text,food_type,local_time,original_logtime,original_logtime_notz,participant,study,time,tz,ID,parsed
0,9bbi18050004,1,2018-07-18,kind bar && coffee,f,9.583333,07-18-2018 09:35AM PDT,2018-07-18 09:35:00,Hero-001,TRE,09:35:00,PDT,0,"[(kind bar, f), (coffee, b)]"
1,9bbi18050004,1,2018-06-07,banana && coffee,f,9.050000,06-07-2018 09:03AM PDT,2018-06-07 09:03:00,Hero-001,TRE,09:03:00,PDT,10,"[(banana, f), (coffee, b)]"


In [41]:
food_dict = defaultdict(lambda : 0)
food_type_dict = defaultdict(lambda : 0)

coffee_backsearch_result = backward_search('coffee')


for i in range(coffee_spot.shape[0]):
    food_lst = coffee_spot.parsed.values[i]
    for pair in food_lst:
        if pair[0] not in coffee_backsearch_result:
            food_dict[pair[0]] += 1
            food_type_dict[pair[1]] += 1

In [42]:
food_count_lst = []
for pair in dict(food_dict).items():
    food_count_lst.append({
        'food': pair[0],
        'count': pair[1]
    })
food_count_df = pd.DataFrame(food_count_lst).sort_values(by = 'count', ascending = False).reset_index(drop = True)
food_count_df

,food,count
0,almond milk,457
1,creamer,360
2,egg,290
3,honey,147
4,almond,143
...,...,...
350,sour cream,1
351,kirkland chocolate bar,1
352,baked veggie,1
353,bagel cream cheese,1


In [43]:
dict(food_type_dict)

{'f': 3041, 'b': 2053, 'm': 51, 'w': 15}

In [44]:
food_count_df.to_csv('food_count_df.csv', index = False)

In [45]:
sample_df.to_csv('sample_df.csv', index = False)

In [46]:
coffee_spot.to_csv('coffee_spot.csv', index = False)